In [1]:
using Laplacians

objc[4619]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[4619]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[4619]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[4619]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgVBLASMatNorm (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
a = readFromFile("../graphs/se_3/graph.mtx");

In [5]:
tree = readFromFile("../graphs/se_3/tree.mtx");

In [6]:
x = readFromFile("../graphs/se_3/x.vec");

In [38]:
# b = readFromFile("../graphs/se_1/b.vec");
b = lap(a) * x;

In [8]:
norm(lap(a) * x - b) / norm(b)

0.0

In [9]:
la = lap(a);

In [10]:
ts = treeSolver(tree)

F = function(b)
    return ts(b - mean(b))
end

(anonymous function)

In [34]:
@time F(b);

  0.205060 seconds (5.75 M allocations: 97.207 MB, 11.59% gc time)


In [39]:
@time myx64 = pcg(la, b, F, tol=1e-3, maxits=100, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)

PCG stopped after: 100 iterations with relative error NaN.
 30.075472 seconds (580.29 M allocations: 10.152 GB, 7.27% gc time)


NaN

In [40]:
dbg = []

for hp in [53,64,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    tsHP = treeSolver(treeHP)

    FHP = function(b)
        return tsHP(b - mean(b))
    end
    
    push!(dbg, "======= $(hp) bit precision")
#     @time myxHP = pcgVMatNorm(laHP, bHP, FHP, xHP, tol=BigFloat(1e-2), maxits=1000, verbose=true);
    @time myxHP = pcgV(laHP, bHP, FHP, tol=BigFloat(1e-3), maxits=100, verbose=true);
    println("Error is: ", Float64(norm(laHP * myxHP - bHP) / norm(bHP)))
    push!(dbg, "")
    push!(dbg, "")
    
end

LoadError: LoadError: InterruptException:
while loading In[40], in expression starting on line 3

In [41]:
writeToFile("../graphs/se_3/_log_julia.txt", dbg)